In [1]:
from deep_orderbook.feeds.coinbase_feed import CoinbaseFeed as Feed
from deep_orderbook.shaper import BookShaper

import asyncio
import pandas as pd
import numpy as np
from pylab import rcParams

In [2]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [3]:
MARKETS = ["BTC-USD", "ETH-USD", "ETH-BTC", ]

In [4]:
async with Feed(markets=MARKETS) as receiver:
    await asyncio.sleep(5)

In [ ]:
b = receiver.depth_managers['BTC-USD'].get_bids()
b = pd.DataFrame(b, columns=['price', 'size']).astype(np.float64).set_index('price')
b.cumsum().iloc[:-1000].plot(title='BTC-USD Bids')

In [ ]:
a = receiver.depth_managers['BTC-USD'].get_asks()
a = pd.DataFrame(a, columns=['price', 'size']).astype(np.float64).set_index('price')
a.cumsum().iloc[:-1000].plot(title=f'BTC-USD Asks')

In [ ]:
receiver.trade_tapes['BTC-USD']

In [ ]:
multi_replay = receiver.multi_generator()
_ = await multi_replay.__anext__()
_

In [ ]:
import aioitertools
genarr = BookShaper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await aioitertools.next(genarr)
_

In [ ]:
genacc = BookShaper.accumulate_array(genarr, markets=MARKETS)
_ = await aioitertools.next(genacc)
_

In [ ]:
async with Feed(markets=MARKETS) as receiver:
    multi_replay = receiver.multi_generator()
    genarr = BookShaper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
    genacc = BookShaper.accumulate_array(genarr, markets=MARKETS)
    async for toshow in BookShaper.images(accumulated_arrays=genacc, every=1, LENGTH=512):
        plt.figure(figsize=(15,11))
        plt.imshow(toshow, origin="upper")
        clear_output(wait=True)
        plt.show()

In [ ]:
#task = asyncio.create_task(images(accumulated_arrays=genacc, every=1, LENGTH=512))